## Upload Images from File Explorer

In [5]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
import io

# Create widgets
uploader = widgets.FileUpload(
    accept='image/*',  # Accepts all image types
    multiple=True  # Allows multiple file selection
)

image_slider = widgets.IntSlider(
    min=0,  # Minimum value for the slider
    max=0,  # Initial maximum value, will be adjusted
    step=1,  # Step count
    description='Image Index:',
    continuous_update=False
)
image_slider.layout.visibility = 'hidden'  # Initially hide the slider

output = widgets.Output()

# A list to store images
stored_images = []

# Function to update slider max and reset its position
def update_slider():
    image_slider.max = len(uploader.value) - 1
    if uploader.value:
        image_slider.value = 0
        image_slider.layout.visibility = 'visible'
    else:
        image_slider.layout.visibility = 'hidden'

# Function to handle file upload changes
def on_files_uploaded(change):
    # Clear existing images from the list
    stored_images.clear()
    # Load new images into the list
    for image_data in uploader.value:
        image = Image.open(io.BytesIO(image_data['content']))
        stored_images.append(image)
    update_slider()
    display_images()

# Function to display the selected image
def display_images(change=None):
    if stored_images:
        image = stored_images[image_slider.value]
        with output:
            clear_output(wait=True)
            display(image)

# Observers
uploader.observe(on_files_uploaded, names='value')
image_slider.observe(display_images, names='value')

# Layout
display(uploader, image_slider, output)


FileUpload(value=(), accept='image/*', description='Upload', multiple=True)

IntSlider(value=0, continuous_update=False, description='Image Index:', layout=Layout(visibility='hidden'), ma…

Output()

## Resize and Crop images

In [6]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image

# Assuming 'stored_images' is already defined and filled with PIL.Image objects

image_slider = widgets.IntSlider(
    min=0,
    max=len(stored_images) - 1 if stored_images else 0,
    step=1,
    description='Image Index:',
    continuous_update=False
)

resize_width = widgets.IntText(
    value=100 if stored_images else 0,
    description='Width:',
    disabled=False
)

resize_height = widgets.IntText(
    value=100 if stored_images else 0,
    description='Height:',
    disabled=False
)

crop_left = widgets.IntSlider(
    value=0,
    min=0,
    max=100,
    description='Crop Left:',
    continuous_update=False
)

crop_right = widgets.IntSlider(
    value=0,
    min=0,
    max=100,
    description='Crop Right:',
    continuous_update=False
)

crop_top = widgets.IntSlider(
    value=0,
    min=0,
    max=100,
    description='Crop Top:',
    continuous_update=False
)

crop_bottom = widgets.IntSlider(
    value=0,
    min=0,
    max=100,
    description='Crop Bottom:',
    continuous_update=False
)

apply_all_button = widgets.Button(
    description='Apply to All',
    disabled=False,
    button_style='',
    tooltip='Apply current settings to all images'
)

output = widgets.Output()

modified_images = []  # To store modified images

def display_images(change=None):
    with output:
        clear_output(wait=True)
        if stored_images:
            image = stored_images[image_slider.value]
            # Resize
            if resize_width.value > 0 and resize_height.value > 0:
                image = image.resize((resize_width.value, resize_height.value))
            # Crop
            left, top, right, bottom = crop_left.value, crop_top.value, crop_right.value, crop_bottom.value
            image = image.crop((left, top, image.width - right, image.height - bottom))
            display(image)

def apply_to_all(b):
    modified_images.clear()
    for img in stored_images:
        # Resize
        if resize_width.value > 0 and resize_height.value > 0:
            resized_img = img.resize((resize_width.value, resize_height.value))
        else:
            resized_img = img
        # Crop
        left, top, right, bottom = crop_left.value, crop_top.value, crop_right.value, crop_bottom.value
        cropped_img = resized_img.crop((left, top, resized_img.width - right, resized_img.height - bottom))
        modified_images.append(cropped_img)
    with output:
        clear_output(wait=True)
        print(f"Applied settings to all {len(modified_images)} images.")

# Observers
image_slider.observe(display_images, names='value')
resize_width.observe(display_images, names='value')
resize_height.observe(display_images, names='value')
crop_left.observe(display_images, names='value')
crop_right.observe(display_images, names='value')
crop_top.observe(display_images, names='value')
crop_bottom.observe(display_images, names='value')
apply_all_button.on_click(apply_to_all)

# Layout
ui = widgets.VBox([
    image_slider,
    widgets.HBox([resize_width, resize_height]),
    widgets.HBox([crop_left, crop_right, crop_top, crop_bottom]),
    apply_all_button,
    output
])

display(ui)


## Filter out Color

In [9]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image, ImageColor
import numpy as np

# Assuming 'modified_images' contains the images with previously applied changes

# Widget to pick the target color
color_picker = widgets.ColorPicker(
    value='red',
    description='Target Color:',
    disabled=False
)

# Slider to set the color distance threshold
threshold_slider = widgets.IntSlider(
    value=255,
    min=0,
    max=255,
    step=1,
    description='Threshold:',
    continuous_update=False
)

# Slider to browse images
image_slider = widgets.IntSlider(
    min=0,
    max=len(modified_images) - 1 if modified_images else 0,
    step=1,
    description='Image Index:',
    continuous_update=False
)

# Output widget for displaying images
output = widgets.Output()

# Button to apply color filter to all images
apply_all_button = widgets.Button(
    description='Apply to All',
    disabled=False,
    button_style='',
    tooltip='Apply color filter settings to all images'
)

filtered_images = []  # To store images after applying the color range filter

def display_filtered_image(change=None):
    with output:
        clear_output(wait=True)
        if modified_images:
            img = modified_images[image_slider.value]
            img = img.resize((500, 500))
            filtered_img = apply_color_filter(img, color_picker.value, threshold_slider.value)
            display(filtered_img)

def apply_color_filter(img, target_color, threshold):
    target_rgb = ImageColor.getrgb(target_color)
    img = img.convert('RGB')
    
    def color_distance(rgb1, rgb2):
        # Calculate Euclidean distance between two RGB values
        return np.sqrt(sum((rgb1[i] - rgb2[i]) ** 2 for i in range(3)))

    pixels = img.load()
    for i in range(img.width):
        for j in range(img.height):
            if color_distance(pixels[i, j], target_rgb) > threshold:
                pixels[i, j] = (0, 0, 0)
    return img

def apply_to_all_images(b):
    filtered_images.clear()
    for img in modified_images:
        filtered_img = apply_color_filter(img, color_picker.value, threshold_slider.value)
        filtered_images.append(filtered_img)
    print(f"Applied color filter to all {len(filtered_images)} images.")

# Observers and event handlers
image_slider.observe(display_filtered_image, names='value')
color_picker.observe(display_filtered_image, names='value')
threshold_slider.observe(display_filtered_image, names='value')
apply_all_button.on_click(apply_to_all_images)

# Layout configuration
ui = widgets.VBox([
    color_picker,
    threshold_slider,
    image_slider,
    apply_all_button,
    output
])

display(ui)


## Detect Edges

In [10]:
import cv2  # OpenCV for edge detection

# Slider to browse images
image_slider = widgets.IntSlider(
    min=0,
    max=len(filtered_images) - 1 if filtered_images else 0,
    step=1,
    description='Image Index:',
    continuous_update=False
)

# Slider for edge detection threshold
edge_threshold_slider = widgets.FloatSlider(
    value=255,
    min=0,
    max=255,
    step=1,
    description='Edge Threshold:',
    continuous_update=False
)

# Output widget for displaying edge-detected images
edge_output = widgets.Output()

def detect_edges(image, threshold):
    # Convert the image to grayscale
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)
    # Apply Canny edge detection
    edges = cv2.Canny(gray, threshold, threshold * 2)
    # Convert edges back to RGB
    edges_rgb = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
    return Image.fromarray(edges_rgb)

def display_edge_image(change=None):
    with edge_output:
        clear_output(wait=True)
        if filtered_images:
            edge_img = detect_edges(filtered_images[image_slider.value], edge_threshold_slider.value)
            display(edge_img)

# Observer for the image slider
image_slider.observe(display_edge_image, names='value')

# Observer for the edge threshold slider
edge_threshold_slider.observe(display_edge_image, names='value')

# Apply edge detection to all images
apply_all_edges_button = widgets.Button(
    description='Apply to All',
    disabled=False,
    button_style='',
    tooltip='Apply edge detection to all images'
)

edge_detected_images = []  # New variable to store edge-detected images

def apply_edge_to_all_images(b):
    global edge_detected_images
    edge_detected_images.clear()  # Clear the existing list
    for img in filtered_images:
        edge_img = detect_edges(img, edge_threshold_slider.value)
        edge_detected_images.append(edge_img)
    print("Edge detection applied to all images.")

# Hook up the event handler for the apply all button
apply_all_edges_button.on_click(apply_edge_to_all_images)

# Layout configuration for edge detection
edge_ui = widgets.VBox([
    image_slider,
    edge_threshold_slider,
    apply_all_edges_button,
    edge_output
])

display(edge_ui)
